In [1]:
! ls /kaggle/input/transformer-models/transformers

PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl
certifi-2023.5.7-py3-none-any.whl
charset_normalizer-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
filelock-3.12.0-py3-none-any.whl
fsspec-2023.5.0-py3-none-any.whl
huggingface_hub-0.15.1-py3-none-any.whl
idna-3.4-py3-none-any.whl
numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
packaging-23.1-py3-none-any.whl
regex-2023.6.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
requests-2.31.0-py3-none-any.whl
safetensors-0.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
tqdm-4.65.0-py3-none-any.whl
transformers-4.30.0.dev0.zip
typing_extensions-4.6.3-py3-none-any.whl
urllib3-2.0.2-py3-none-any.whl


In [2]:
!pip install transformers --no-index --find-links=file:///kaggle/input/transformer-models/transformers/

Looking in links: file:///kaggle/input/transformer-models/transformers/


In [3]:
import pandas as pd
import io
import torch
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor
import requests
from PIL import Image
import re

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
def display_deplot_output(deplot_output, visualize=True):
    '''
    The raw output of deplot
    TITLE | Rural population (%) long-run with 2050 projections<0x0A>(OWID) in Greece<0x0A>Years | Rural population<0x0A>1940 | 47.38<0x0A>1960 | 43.68<0x0A>1980 | 30.28<0x0A>...
    '''
    
    deplot_output = deplot_output.replace("<0x0A>", "\n").replace(" | ", "\t")

    second_a_index = [m.start() for m in re.finditer('\t', deplot_output)][1]
    last_newline_index = deplot_output.rfind('\n', 0, second_a_index) 

    title = deplot_output[:last_newline_index]
    table = deplot_output[last_newline_index+1:]

    data = io.StringIO(table)
    df = pd.read_csv(data, sep='\t')
    if visualize:
        display(df)
    
    return df
    

def deplot(path, model, processor, device, visualize=True):

    image = Image.open(path)
    
    if visualize:
        display(image)
    #problem is here
    inputs = processor(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt")

    # Move inputs to GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}

    predictions = model.generate(**inputs, max_new_tokens=512)
    return processor.decode(predictions[0], skip_special_tokens=True)

In [5]:
deplot_weights_path = "/kaggle/input/google-deplot-model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
deplot_model = Pix2StructForConditionalGeneration.from_pretrained(deplot_weights_path).to(device)
processor = Pix2StructProcessor.from_pretrained(deplot_weights_path)

In [6]:
device

device(type='cuda')

In [7]:
deplot_model

Pix2StructForConditionalGeneration(
  (encoder): Pix2StructVisionModel(
    (embeddings): Pix2StructVisionEmbeddings(
      (patch_projection): Linear(in_features=768, out_features=768, bias=True)
      (row_embedder): Embedding(4096, 768)
      (column_embedder): Embedding(4096, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Pix2StructVisionEncoder(
      (layer): ModuleList(
        (0-11): 12 x Pix2StructVisionLayer(
          (attention): Pix2StructVisionAttention(
            (query): Linear(in_features=768, out_features=768, bias=False)
            (key): Linear(in_features=768, out_features=768, bias=False)
            (value): Linear(in_features=768, out_features=768, bias=False)
            (output): Linear(in_features=768, out_features=768, bias=False)
          )
          (mlp): Pix2StructVisionMlp(
            (wi_0): Linear(in_features=768, out_features=2048, bias=False)
            (wi_1): Linear(in_features=768, out_features=2048, bias=False)
 

In [8]:
processor

Pix2StructProcessor:
- image_processor: Pix2StructImageProcessor {
  "do_convert_rgb": true,
  "do_normalize": true,
  "image_processor_type": "Pix2StructImageProcessor",
  "is_vqa": true,
  "max_patches": 2048,
  "patch_size": {
    "height": 16,
    "width": 16
  },
  "processor_class": "Pix2StructProcessor"
}

- tokenizer: T5TokenizerFast(name_or_path='/kaggle/input/google-deplot-model', vocab_size=50344, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<

In [9]:
def deplot_inference(image_path, visualize):
    deplot_output=deplot(image_path, deplot_model, processor, device, visualize)
    return display_deplot_output(deplot_output, visualize)

# Classification Model

In [10]:
import torch
import torchvision
import torch.utils.data
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import cv2
import numpy as np

In [11]:
label_map = {'dot': 0, 'horizontal_bar' : 1, 'vertical_bar': 2, 'line': 3, 'scatter': 4}
label_idx_to_classname = {v: k for k, v in label_map.items()}

In [12]:
classification_model = torchvision.models.resnet50(pretrained=False)

num_features = classification_model.fc.in_features
classification_model.fc = nn.Linear(num_features, 5)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
classification_model = classification_model.to(device)
classification_model.eval()

state_dict = torch.load("/kaggle/input/simple-submission-classification-task-training/Benetech _ResNet50_fold0.pth")
classification_model.load_state_dict(state_dict)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [13]:
val_transforms = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

def classification_inference(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(500,300))
    img = img.astype(np.float32)/255.0
    img = val_transforms(img)
    
#     print(img.shape)
    
#     inp = torch.stack([img.permute(1, 2, 0)]).cuda()
    inp = torch.stack([img]).cuda()
    out = classification_model(inp).detach().cpu().numpy()
    return label_idx_to_classname[np.argmax(out)]

# Inference pipeline

In [14]:
import os
from tqdm import tqdm
import math

IMAGE_FOLDER = "/kaggle/input/benetech-making-graphs-accessible/test/images"

all_ids = []
all_values = []
all_chart_types = []

for image_name in tqdm(os.listdir(IMAGE_FOLDER)):
    if ".jpg" not in image_name:
        continue
    try:
        image_path = os.path.join(IMAGE_FOLDER, image_name)
        graph_type = classification_inference(image_path)
        inference_df = deplot_inference(image_path, False)
        
        x_values = inference_df[inference_df.columns[0]].values
        y_values = inference_df[inference_df.columns[1]].values
        
        # check value type for each graph type
        if graph_type in ["horizontal_bar", "vertical_bar", "line", "dot", "scatter"]:
            # x is categorical, y is numerical
            formatted_y_values = []
            for v in y_values:
                try:
                    float(v)
                    if math.isnan(float(v)):
                        formatted_y_values.append(0)
                    else:
                        formatted_y_values.append(v)
                except:
                    formatted_y_values.append(0)
                    
            y_values = formatted_y_values
        
        if graph_type in ["dot", "scatter"]:
            # for dot graph, x could be categorical as well as numerical, how to handle this?
            # just leave all the value be numerical for now

            formatted_x_values = []
            for v in x_values:
                try:
                    float(v)
                    if math.isnan(float(v)):
                        formatted_x_values.append(0)
                    else:
                        formatted_x_values.append(v)
                except:
                    formatted_x_values.append(0)
                    
            x_values = formatted_x_values
            
        
        length = min(len(x_values), len(y_values))
        x_values = ";".join([str(v).strip() for v in x_values][:length])
        y_values = ";".join([str(v).strip() for v in y_values][:length])
    except Exception as e:
        print("Exception", e)
        graph_type = "line"
        x_values = "0;0"
        y_values = "0;0"
    
    image_id = image_name.split(".")[0]

    all_ids.append(image_id + "_x")
    all_values.append(x_values)
    
    all_ids.append(image_id + "_y")
    all_values.append(y_values)
    
    all_chart_types.extend([graph_type, graph_type])

 20%|██        | 1/5 [00:24<01:37, 24.43s/it]

Exception Connection error, and we cannot find the requested files in the disk cache. Please try again or make sure your Internet connection is on.


 40%|████      | 2/5 [00:44<01:05, 21.85s/it]

Exception Connection error, and we cannot find the requested files in the disk cache. Please try again or make sure your Internet connection is on.


 60%|██████    | 3/5 [01:04<00:42, 21.03s/it]

Exception Connection error, and we cannot find the requested files in the disk cache. Please try again or make sure your Internet connection is on.


 80%|████████  | 4/5 [01:24<00:20, 20.64s/it]

Exception Connection error, and we cannot find the requested files in the disk cache. Please try again or make sure your Internet connection is on.


100%|██████████| 5/5 [01:44<00:00, 20.92s/it]

Exception Connection error, and we cannot find the requested files in the disk cache. Please try again or make sure your Internet connection is on.


In [15]:
submission_df = pd.DataFrame({
    "id": all_ids,
    "data_series": all_values,
    "chart_type": all_chart_types
})

submission_df.to_csv("submission.csv", index=False)


In [16]:
display(submission_df)

,id,data_series,chart_type
0,000b92c3b098_x,0;0,line
1,000b92c3b098_y,0;0,line
2,01b45b831589_x,0;0,line
3,01b45b831589_y,0;0,line
4,00f5404753cf_x,0;0,line
5,00f5404753cf_y,0;0,line
6,00dcf883a459_x,0;0,line
7,00dcf883a459_y,0;0,line
8,007a18eb4e09_x,0;0,line
9,007a18eb4e09_y,0;0,line
